## Getting access to training data

In [ ]:
from stravalib import Client
import pickle
import time
import pandas as pd
import datetime
import matplotlib
import matplotlib.pyplot as plt

client = Client()

MY_STRAVA_CLIENT_ID, MY_STRAVA_CLIENT_SECRET = open('../access/client.secret').read().strip().split(',')
print ('Client ID and secret read from file'.format(MY_STRAVA_CLIENT_ID) )

## Getting access token

In [ ]:
with open('../access/access_token.pickle', 'rb') as f:
    access_token = pickle.load(f)
    
print('Latest access token read from file:')
access_token

In [ ]:
if time.time() > access_token['expires_at']:
    print('Token has expired, will refresh')
    refresh_response = client.refresh_access_token(client_id=MY_STRAVA_CLIENT_ID, 
                                               client_secret=MY_STRAVA_CLIENT_SECRET, 
                                               refresh_token=access_token['refresh_token'])
    access_token = refresh_response
    with open('../access/access_token.pickle', 'wb') as f:
        pickle.dump(refresh_response, f)
    print('Refreshed token saved to file')

    client.access_token = refresh_response['access_token']
    client.refresh_token = refresh_response['refresh_token']
    client.token_expires_at = refresh_response['expires_at']
        
else:
    print('Token still valid, expires at {}'
          .format(time.strftime("%a, %d %b %Y %H:%M:%S %Z", time.localtime(access_token['expires_at']))))

    client.access_token = access_token['access_token']
    client.refresh_token = access_token['refresh_token']
    client.token_expires_at = access_token['expires_at']

## Collect activities in DataFrame

In [ ]:
activities = client.get_activities(limit=5000)

In [ ]:
my_cols =['name',
          'start_date_local',
          'type',
          'distance',
          'moving_time',
          'elapsed_time',
          'total_elevation_gain',
          'elev_high',
          'elev_low',
          'average_speed',
          'max_speed',
          'average_heartrate',
          'max_heartrate',
          'start_latitude',
          'start_longitude']

In [ ]:
data = []
for activity in activities:
    my_dict = activity.to_dict()
    data.append([activity.id]+[my_dict.get(x) for x in my_cols])
    
# Add id to the beginning of the columns, used when selecting a specific activity
my_cols.insert(0,'id')

In [ ]:
df = pd.DataFrame(data, columns=my_cols)
df

## Add training types to activities

| Training type | Activity types |
| ------------ | ------------ |
| C (Climbing) | RockClimbing |
| AE (Aerobic endurance)| Ride, VirtualRide, Hike, Run, AlpineSki, BackcountrySki, Canoeing |
| CE/S (Strength Training)| WeightTraining, Workout |
| Recovery | Walk, Yoga |


In [ ]:
df['type'].value_counts()

In [ ]:
training_type_dict  = {'RockClimbing': 'C', 'Ride': 'AE', 'VirtualRide': 'AE', 'Hike': 'AE', 'Run': 'AE', 'AlpineSki': 'AE', 'BackcountrySki': 'AE', 'WeightTraining': 'CE/S', 'Workout': 'CE/S', 'Walk': 'Recovery', 'Yoga': 'Recovery', 'Canoeing': 'AE'}
df['training_type'] = df['type'].apply(lambda x: training_type_dict.get(x))
df['training_type'].value_counts()

In [ ]:
df['day'] = df['start_date_local'].apply(lambda x: x.split(sep='T')[0])

df['week'] = df['day'].apply(lambda x: datetime.datetime.fromisoformat(x).isocalendar()[1])
df['year'] = df['day'].apply(lambda x: datetime.datetime.fromisoformat(x).isocalendar()[0])

start_date_index = pd.DatetimeIndex(df['start_date_local'])
df_timeindex = df.drop(columns='start_date_local')
df_timeindex = df_timeindex.set_index(start_date_index)

### Progress in last weeks

In [ ]:
df_weeksum = df_timeindex[['distance', 'moving_time', 'total_elevation_gain']].resample('W').sum()
df_weeksum

In [ ]:
df_vis = df_weeksum
df_vis['moving_time_time_delta'] = df_vis['moving_time'].apply(lambda x: datetime.timedelta(seconds=x))
df_vis['moving_time_time_delta']

### Visualisation

In [ ]:
from matplotlib.ticker import FuncFormatter

def format_func(x, pos):
    hours = int(x//3600)
    minutes = int((x%3600)//60)
    seconds = int(x%60)

    return "{:d}:{:02d}".format(hours, minutes)
    # return "{:d}:{:02d}:{:02d}".format(hours, minutes, seconds)

formatter = FuncFormatter(format_func)


df_vis = df_weeksum.tail(104)
# df_vis['moving_time_time_delta'] = df_vis['moving_time'].apply(lambda x: datetime.timedelta(seconds=x))

fig, axes = plt.subplots(3, figsize = (9, 9))

axes[0].bar(df_vis.index, df_vis.distance, width=7, align='edge', edgecolor='black', label='Total distance')
axes[0].plot(df_vis.index, df_vis.distance.rolling(4).mean(), color='red', label='Moving average')
axes[0].legend(loc='upper left')

axes[1].bar(df_vis.index, df_vis.moving_time, width=7, align='edge', edgecolor='black', label='Moving time')
axes[1].plot(df_vis.index, df_vis.moving_time.rolling(4).mean(), color='red', label='Moving average')
axes[1].yaxis.set_major_formatter(formatter)
axes[1].yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=10800))
axes[1].legend(loc='upper left')

axes[2].bar(df_vis.index, df_vis.total_elevation_gain, width=7, align='edge', edgecolor='black', label='Total elevation gain')
axes[2].plot(df_vis.index, df_vis.total_elevation_gain.rolling(4).mean(), color='red', label='Moving average')
axes[2].legend(loc='upper left')

### Summary of last week

In [ ]:

last_week = datetime.date.today() - datetime.timedelta(days=7)
year = last_week.isocalendar()[0]
week_number = last_week.isocalendar()[1]

# Specify a different week here
# year = 2023
# week_number = 15

last_monday = datetime.datetime.strptime(f'{year}-{week_number}-1', "%Y-%W-%w").date()
last_monday_str = str(last_monday)
last_sunday = last_monday + datetime.timedelta(days=6)
last_sunday_str = str(last_sunday)

print('Training data between', last_monday, 'and', last_sunday)

df_lastweek = df.query('@last_monday_str <= day <= @last_sunday_str')

In [ ]:
# Sum of training metrics

df_lastweek[['distance', 'moving_time', 'total_elevation_gain', 'training_type']].groupby('training_type').sum()

In [ ]:
# Median of training metrics

df_lastweek[['average_heartrate', 'max_heartrate', 'average_speed', 'max_speed', 'training_type']].groupby('training_type').median()